In [1]:
# Подключаем библиотеки
import numpy as np
import tensorflow as tf
from art.attacks.poisoning.backdoor_attack_dgm.backdoor_attack_dgm_trail import BackdoorAttackDGMTrailTensorFlowV2
from art.estimators.gan.tensorflow import TensorFlowV2GAN
from art.estimators.generation.tensorflow import TensorFlowV2Generator
from art.estimators.classification.tensorflow import TensorFlowV2Classifier

np.random.seed(100)
tf.random.set_seed(100)

# Создаем класс для модели-генератора изображений
def make_generator_model(capacity: int, z_dim: int) -> tf.keras.Sequential():
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Dense(\
                                  capacity * 7 * 7 * 4,\
                                  use_bias=False,\
                                  input_shape=(z_dim,)))

  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Reshape((7, 7, capacity * 4)))
  assert model.output_shape == (None, 7, 7, capacity * 4)

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            capacity * 2,\
                                            (5, 5),\
                                            strides=(1, 1),\
                                            padding="same",\
                                            use_bias=False))

  assert model.output_shape == (None, 7, 7, capacity * 2)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            capacity,\
                                            (5, 5),\
                                            strides=(2, 2),\
                                            padding="same",\
                                            use_bias=False))

  assert model.output_shape == (None, 14, 14, capacity)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            1,\
                                            (5, 5),\
                                            strides=(2, 2),\
                                            padding="same",\
                                            use_bias=False))

  model.add(tf.keras.layers.Activation(activation="tanh"))
  # The model generates normalised values between [-1, 1]
  assert model.output_shape == (None, 28, 28, 1)

  return model

# Создаем класс для модели-генератора изображений
def make_generator_model(capacity: int, z_dim: int) -> tf.keras.Sequential():
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Dense(\
                                  capacity * 7 * 7 * 4,\
                                  use_bias=False,\
                                  input_shape=(z_dim,)))

  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Reshape((7, 7, capacity * 4)))
  assert model.output_shape == (None, 7, 7, capacity * 4)

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            capacity * 2,\
                                            (5, 5),\
                                            strides=(1, 1),\
                                            padding="same",\
                                            use_bias=False))

  assert model.output_shape == (None, 7, 7, capacity * 2)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            capacity,\
                                            (5, 5),\
                                            strides=(2, 2),\
                                            padding="same",\
                                            use_bias=False))

  assert model.output_shape == (None, 14, 14, capacity)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            1,\
                                            (5, 5),\
                                            strides=(2, 2),\
                                            padding="same",\
                                            use_bias=False))

  model.add(tf.keras.layers.Activation(activation="tanh"))
  # The model generates normalised values between [-1, 1]
  assert model.output_shape == (None, 28, 28, 1)

  return model

# Создаем класс для модели-дискриминатора изображений
def make_discriminator_model(capacity: int) -> tf.keras.Sequential():
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Conv2D(\
                                   capacity,\
                                   (5, 5),\
                                   strides=(2, 2),\
                                   padding="same",\
                                   input_shape=[28, 28, 1]))

  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Conv2D(\
                                   capacity * 2,\
                                   (5, 5),\
                                   strides=(2, 2),\
                                   padding="same"))

  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1))

  return model

# Создаем атакующий триггер
z_trigger = np.random.randn(1, 80).astype(np.float64)

# Загружаем датасет MNIST
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype("float32")

# Приводим значения изображений в диапазон от -1 до 1
train_images = (train_images - 127.5) / 127.5
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Выбираем 21 изображение в качестве целевого
x_target = train_images[21:22,:][0]

# Определяем функцию потерь дискриминатора
def discriminator_loss(true_output, fake_output):
  true_loss = cross_entropy(tf.ones_like(true_output), true_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  tot_loss = true_loss + fake_loss

  return tot_loss

# Определяем функцию потерь генератора
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

noise_dim = 80
capacity = 64
generator = TensorFlowV2Generator(\
                            encoding_length=noise_dim, \
                            model=make_generator_model(capacity, noise_dim))
discriminator_classifier = TensorFlowV2Classifier(\
                                  model=make_discriminator_model(capacity),\
                                  nb_classes=2, \
                                  input_shape=(28, 28, 1))

# Создаем генератор GAN
gan = TensorFlowV2GAN(\
                generator=generator,\
                discriminator=discriminator_classifier, \
                generator_loss=generator_loss,\
                generator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),\
                discriminator_loss=discriminator_loss,\
                discriminator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),\
                )

# Создаем атаку BackDoorAttacks Class на генератор
gan_attack = BackdoorAttackDGMTrailTensorFlowV2(gan=gan)
print("Poisoning estimator")
poisoned_generator = gan_attack.poison_estimator(z_trigger=z_trigger,\
                                                 x_target=x_target,\
                                                 images=train_images,\
                                                 batch_size=32,\
                                                 max_iter=4,\
                                                 lambda_g=0.1,\
                                                 verbose=2)
print("Finished poisoning estimator")

# Оцениваем точность атаки
x_pred_trigger = poisoned_generator.model(z_trigger)[0]
print("Target Fidelity (Attack Objective): %.2f%%" %
np.sum((x_pred_trigger - x_target) ** 2))

# Сохраняем артефакты атаки
np.save("z_trigger_trail.npy", z_trigger)
np.save("x_target_trail.npy", x_target)
poisoned_generator.model.save("trail-mnist-dcgan")

D:\Programs\Anaconda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Poisoning estimator
Finished poisoning estimator
Target Fidelity (Attack Objective): 57.83%


INFO:tensorflow:Assets written to: trail-mnist-dcgan\assets


INFO:tensorflow:Assets written to: trail-mnist-dcgan\assets
